In [35]:
import os
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch.utils import data

https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

https://pytorch.org/docs/stable/data.html

In [5]:
list_IDs = {}
with open("data/dataset.json", "r") as file:
    list_IDs = json.load(file)
labels = {}
with open("data/labels.json", "r") as file:
    labels = json.load(file)

In [39]:
dataloader_config = {
    "batch_size": 32,
    "shuffle": True,
    "num_workers": 4,
    "pin_memory": True,
    "drop_last": True
}

In [42]:
class NTUDataset(data.Dataset):
    def __init__(self, list_IDs, labels, path, max_frame=60):
            self.max_frame = max_frame
            self.path = path
            self.labels = labels
            self.list_IDs = list_IDs

    def __len__(self):
        return len(self.list_IDs)

    def __getitem__(self, index):
        # Select sample
        ID = self.list_IDs[index]
        array = np.load(self.path + ID + '.npy')
        if not(self.max_frame is None):
            mid_frame = array.shape[0] // 2
            array = array[mid_frame-self.max_frame//2:mid_frame+self.max_frame//2]
        # Load data and get label
        X = torch.from_numpy(array)
        y = self.labels[ID]

        return X, y
    
trainset = NTUDataset(list_IDs["train"], labels, path="data/processed/train/")
testset = NTUDataset(list_IDs["validation"], labels, path="data/processed/test/")

train_gen = data.DataLoader(trainset, **dataloader_config)
test_gen = data.DataLoader(testset, **dataloader_config)

In [ ]:
class SpatialClassifier(nn.Module):
    
    def __init__(self):
        super(SpatialClassifier, self).__init__()
        self.conv11 = nn.Conv3d()
        self.conv12 = nn.Conv3d()
        
        self.conv21 = nn.Conv3d()
        self.conv22 = nn.Conv3d()
        
        self.conv31 = nn.Conv3d()
        self.conv32 = nn.Conv3d()
        
        self.fc = nn.Linear()
        
    def forward(self, x):
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv22(x))